In [2]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
import os
import django
import asyncio
from asgiref.sync import sync_to_async

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "songpicker.settings")
django.setup()

from songs.models import Song

# 동기 함수로 데이터 가져오기
def get_songs():
    return list(Song.objects.all().values())

# 비동기 함수 정의
async def async_get_songs():
    return await sync_to_async(get_songs)()

# Jupyter Notebook에서 비동기 함수 실행을 위한 헬퍼 함수
async def run_async():
    return await async_get_songs()

# 이 셀을 실행하여 데이터 가져오기
songs = await run_async()

In [11]:
df = pd.DataFrame.from_records(songs)

In [12]:
# 데이터 확인
print(df.head())
print(df.columns)

   id  acousticness  bpm composer  \
0   1            40   99      위종수   
1   2            42  120      MGR   
2   3            75  127      강승원   
3   4             0  144      이근상   
4   5             4  138      김창환   

                                         cover_image  danceability  energy  \
0  https://i.scdn.co/image/ab67616d0000b2736fa3ae...            67      60   
1  https://i.scdn.co/image/ab67616d0000b2730d1a65...            46      50   
2  https://i.scdn.co/image/ab67616d0000b2738b125e...            46      25   
3  https://i.scdn.co/image/ab67616d0000b273472495...            38      64   
4  https://i.scdn.co/image/ab67616d0000b273f59599...            73      96   

    genre  happiness  is_popular lyricist  \
0     발라드         49           1      김태훈   
1     발라드         16           1      양재선   
2  포크/블루스         15           1      강승원   
3    록/메탈         26           1  박준배,이민욱   
4      댄스         86           1      김창환   

                                     

In [13]:
# 필요한 특성 선택
features = ['bpm', 'energy', 'danceability', 'happiness', 'acousticness']

In [14]:
# 'singer'와 'tune' 원-핫 인코딩
df_encoded = pd.get_dummies(df, columns=['singer', 'tune'])

In [15]:
# 'released_at'에서 년도 추출
df_encoded['release_year'] = pd.to_datetime(df_encoded['released_at']).dt.year

In [16]:
# 최종 특성 목록 업데이트
features += [col for col in df_encoded.columns if col.startswith('singer_') or col.startswith('tune_')]
features.append('release_year')

In [26]:
from sklearn.preprocessing import StandardScaler

def get_recommendations_from_multiple_songs(song_numbers, df_original=df, df_encoded=df_encoded, n_recommendations=20):
    # 필요한 특성 선택
    features = ['bpm', 'energy', 'danceability', 'happiness', 'acousticness']
    features += [col for col in df_encoded.columns if col.startswith('singer_') or col.startswith('tune_')]
    features.append('release_year')
    
    # 특성 데이터 준비
    X = df_encoded[features]
    
    # 특성 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 입력된 곡들의 인덱스 찾기
    indices = [df_encoded.index[df_encoded['number'] == song_number].tolist()[0] for song_number in song_numbers]
    
    # 입력된 곡들의 특성 벡터 평균 계산
    avg_features = np.mean(X_scaled[indices], axis=0)
    
    # 모든 곡과의 유사도 계산
    sim_scores = list(enumerate(cosine_similarity([avg_features], X_scaled)[0]))
    
    # 유사도에 따라 곡들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 입력된 곡들 제외
    sim_scores = [score for score in sim_scores if score[0] not in indices]
    
    # 가장 유사한 n_recommendations개의 곡 선택
    sim_scores = sim_scores[:n_recommendations]
    
    # 선택된 곡들의 인덱스 추출
    song_indices = [i[0] for i in sim_scores]
    
    # 추천 곡 정보 반환
    recommended_songs = df_original.iloc[song_indices][['number', 'title', 'singer']]
    recommended_songs['similarity_score'] = [score[1] for score in sim_scores]
    
    return recommended_songs

# 함수 사용 예시
input_song_numbers = [27615, 9290, 4448, 64011, 44253]  # 예시로 5개의 곡 number를 입력
recommendations = get_recommendations_from_multiple_songs(input_song_numbers)

print(recommendations)

     number                                           title           singer  \
187    9034                                         그래서 그대는          얀(Yarn)   
40     1479                                           사랑했지만              김광석   
33    48367                        너의 모든 순간(드라마 "별에서 온 그대")              성시경   
416   76514                                            LADY             米津玄師   
422   44855                         KICK BACK ("チェンソ?マン"OP)             米津玄師   
593   44591                                感電 (ドラマ"MIU404")             米津玄師   
487    3303                                             뽀뽀뽀               동요   
410   40993  Endless Rain ("X2","乙?CONNECTION"OST,映?"ジパング")          X JAPAN   
474   45657                                     투지(만화"나루토")               버즈   
257   66685                                        Timeless            SG워너비   
162    9431                                           소주 한잔              임창정   
9     85183                             

In [21]:
print(df.columns)

      id  acousticness  bpm   composer  \
0      1            40   99        위종수   
1      2            42  120        MGR   
2      3            75  127        강승원   
3      4             0  144        이근상   
4      5             4  138        김창환   
..   ...           ...  ...        ...   
597  598            82  122         8s   
598  599             0   96  音羽-otoha-   
599  600             3  118        なとり   
600  601             0  123       ?藤英雅   
601  602            62   99        이수인   

                                           cover_image  danceability  energy  \
0    https://i.scdn.co/image/ab67616d0000b2736fa3ae...            67      60   
1    https://i.scdn.co/image/ab67616d0000b2730d1a65...            46      50   
2    https://i.scdn.co/image/ab67616d0000b2738b125e...            46      25   
3    https://i.scdn.co/image/ab67616d0000b273472495...            38      64   
4    https://i.scdn.co/image/ab67616d0000b273f59599...            73      96   
..             

In [27]:
from sklearn.neighbors import NearestNeighbors

In [37]:
X = df_encoded[features]

# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# KNN 모델 초기화 및 학습
n_neighbors = 25  # 추천할 곡 수 + 입력 곡 수
knn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
knn.fit(X_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=25)

In [50]:
def get_recommendations_knn(song_numbers, df, df_encoded, X_scaled, knn, n_recommendations=20):
    # 입력된 곡들의 인덱스 찾기
    input_indices = [df[df['number'] == song_number].index[0] for song_number in song_numbers]
    
    # 입력된 곡들의 특성 벡터 평균 계산
    avg_features = np.mean(X_scaled[input_indices], axis=0).reshape(1, -1)
    
    # 가장 가까운 이웃 찾기 (자기 자신 포함)
    distances, indices = knn.kneighbors(avg_features, n_neighbors=len(X_scaled))
    
    # 입력된 곡들 제외 및 상위 n_recommendations개 선택
    recommended_indices = [idx for idx in indices[0] if idx not in input_indices][:n_recommendations]
    
    # 추천 곡 정보 반환
    recommended_songs = df.iloc[recommended_indices][['number', 'title', 'singer']]
    recommended_songs['distance'] = [distances[0][list(indices[0]).index(idx)] for idx in recommended_indices]
    
    return recommended_songs.sort_values('distance')

In [53]:
# 테스트
input_song_numbers = [27615, 9290, 4448, 64011, 44253]  # 예시로 5개의 곡 number를 입력
recommendations = get_recommendations_knn(input_song_numbers, df, df_encoded, X_scaled, knn)
print("Recommended Songs:")
print(recommendations)

Recommended Songs:
     number        title singer   distance
472   82295        생일 노래     동요  10.602048
500     138        고향의 봄     동요  10.690450
458    5566       곰 세 마리     동요  10.809642
487    3303          뽀뽀뽀     동요  10.828838
482   60126    머리 어깨 무릎발     동요  10.835696
465   87327      멋쟁이 토마토     동요  10.860016
468   63073  네잎 클로버(5학년)     동요  10.921990
475    1638         작은 별     동요  10.940204
489   87337       작은 동물원     동요  11.050823
467    1018          산토끼     동요  11.054910
601     303      둥글게 둥글게     동요  11.063622
473   68587     아빠! 힘내세요     동요  11.120346
483    4049        아기 염소     동요  11.142561
494    1618          구슬비     동요  11.202873
466    3314         학교 종     동요  11.209598
498    4599      코끼리 아저씨     동요  11.230427
486    4506  엄마 돼지 아기 돼지     동요  11.488142
476    3093          솜사탕     동요  11.606651
480    4543       예쁜 아기곰     동요  11.669083
493    1026       얼룩 송아지     동요  11.690214


In [40]:
# 입력된 곡들의 정보 출력
print("\nInput Songs:")
input_songs = df[df['number'].isin(input_song_numbers)][['number', 'title', 'singer']]
print(input_songs)


Input Songs:
     number                  title   singer
0     27615  아로하(드라마 "슬기로운 의사 생활")      조정석
1      9290                     희재      성시경
2      4448                 서른 즈음에      김광석
3     64011                      심  얀(Yarn)
409   44253   Lemon (ドラマ"アンナチュラル")     米津玄師


In [41]:
# 추천된 곡들과 입력 곡들의 특성 비교
print("\nFeature Comparison:")
input_features = X.iloc[[df_encoded.index[df_encoded['number'] == num].tolist()[0] for num in input_song_numbers]]
recommended_features = X.iloc[recommendations.index]

input_mean = input_features.mean()
recommended_mean = recommended_features.mean()

comparison = pd.DataFrame({
    'Input Mean': input_mean,
    'Recommended Mean': recommended_mean
})
print(comparison)


Feature Comparison:
               Input Mean  Recommended Mean
bpm                 115.4            118.15
energy               53.0             62.45
danceability         50.0             53.60
happiness            30.2             37.80
acousticness         38.8             33.20
...                   ...               ...
tune_F# Major         0.0              0.10
tune_F# Minor         0.0              0.00
tune_G Major          0.0              0.10
tune_G Minor          0.0              0.00
release_year       2008.6           2008.60

[442 rows x 2 columns]


In [42]:
# 데이터프레임 정보 출력
print("\nDataFrame Info:")
print(df.info())
print("\nEncoded DataFrame Info:")
print(df_encoded.info())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            602 non-null    int64 
 1   acousticness  602 non-null    int64 
 2   bpm           602 non-null    int64 
 3   composer      602 non-null    object
 4   cover_image   602 non-null    object
 5   danceability  602 non-null    int64 
 6   energy        602 non-null    int64 
 7   genre         602 non-null    object
 8   happiness     602 non-null    int64 
 9   is_popular    602 non-null    int64 
 10  lyricist      602 non-null    object
 11  lyrics        602 non-null    object
 12  number        602 non-null    int64 
 13  released_at   602 non-null    object
 14  singer        602 non-null    object
 15  title         602 non-null    object
 16  tune          602 non-null    object
dtypes: int64(8), object(9)
memory usage: 80.1+ KB
None

Encoded DataFrame Info:

In [43]:
# 샘플 데이터 출력
print("\nSample Data:")
print(df.head())
print("\nSample Encoded Data:")
print(df_encoded.head())


Sample Data:
   id  acousticness  bpm composer  \
0   1            40   99      위종수   
1   2            42  120      MGR   
2   3            75  127      강승원   
3   4             0  144      이근상   
4   5             4  138      김창환   

                                         cover_image  danceability  energy  \
0  https://i.scdn.co/image/ab67616d0000b2736fa3ae...            67      60   
1  https://i.scdn.co/image/ab67616d0000b2730d1a65...            46      50   
2  https://i.scdn.co/image/ab67616d0000b2738b125e...            46      25   
3  https://i.scdn.co/image/ab67616d0000b273472495...            38      64   
4  https://i.scdn.co/image/ab67616d0000b273f59599...            73      96   

    genre  happiness  is_popular lyricist  \
0     발라드         49           1      김태훈   
1     발라드         16           1      양재선   
2  포크/블루스         15           1      강승원   
3    록/메탈         26           1  박준배,이민욱   
4      댄스         86           1      김창환   

                       

In [44]:
print("Rows with NaN in df:")
print(df[df.isnull().any(axis=1)])

print("\nRows with NaN in df_encoded:")
print(df_encoded[df_encoded.isnull().any(axis=1)])

Rows with NaN in df:
Empty DataFrame
Columns: [id, acousticness, bpm, composer, cover_image, danceability, energy, genre, happiness, is_popular, lyricist, lyrics, number, released_at, singer, title, tune]
Index: []

Rows with NaN in df_encoded:
Empty DataFrame
Columns: [id, acousticness, bpm, composer, cover_image, danceability, energy, genre, happiness, is_popular, lyricist, lyrics, number, released_at, title, singer_#안녕, singer_& Marketa Irglova, singer_(여자)아이들, singer_10-FEET, singer_10cc, singer_10cm, singer_164 feat.GUMI, singer_40, singer_??色社?, singer_?わらの一味, singer_?永英明, singer_?生たかお, singer_ABBA, singer_Adele, singer_Ado, singer_Aimer, singer_Alicia Keys, singer_Anne-Marie, singer_Avril Lavigne, singer_BANK, singer_BE'O, singer_BIG MAMA, singer_BON JOVI, singer_BROWN EYES, singer_BUCK, singer_Backstreet.., singer_Beatles, singer_Bee Gees, singer_Benson Boone, singer_Beyonce, singer_Beyonce Knowles, singer_Billy Joel, singer_Bobby Kim, singer_Bon Jovi, singer_Boyz II Men, singe